# M5 Forecasting Competition GluonTS 

## Attention: MXNet 1.4.1 and GluonTS 0.4.2 must be used!

### Imports


In [1]:
%matplotlib inline
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from tqdm.autonotebook import tqdm
from pathlib import Path
mx.random.seed(42)
ctx = mx.gpu(0)

/home/tom/anaconda3/envs/mxnet/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  if __name__ == '__main__':


We also define globally accessible variables, such as the prediction length and the input path for the M5 data. Note that `single_prediction_length` corresponds to the length of the validation/evaluation periods, while `submission_prediction_length` corresponds to the length of both these periods combined.



In [2]:
single_prediction_length = 28
submission_prediction_length = single_prediction_length*2 
prediction_length = submission_prediction_length
m5_input_path="./input"


### Reading the M5 data into GluonTS

First we need to convert the provided M5 data into a format that is readable by GluonTS. At this point we assume that the M5 data, which can be downloaded from Kaggle, is present under `m5_input_path`.

In [3]:
sales_train_evaluation = pd.read_csv(f'{m5_input_path}/sales_train_evaluation.csv')
sales_train_validation = pd.read_csv(f'{m5_input_path}/sales_train_validation.csv')
sample_submission = pd.read_csv(f'{m5_input_path}/sample_submission.csv')
sell_prices = pd.read_csv(f'{m5_input_path}/sell_prices.csv')

We start the data convertion process by building dynamic features (features that change over time, just like the target values). Here, we are mainly interested in the event indicators `event_type_1` and `event_type_2`. We will mostly drop dynamic time features as GluonTS will automatically add some of these as part of many models' transformation chains.

In [4]:

cal_ = pd.read_csv(f'{m5_input_path}/calendar.csv', parse_dates=['date']).fillna(0)

cal = cal_.drop(
                ['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'event_name_2', 'event_type_2', 'd'], 
                axis=1
            ).astype('category')

for col in cal:
    cal[col] = cal[col].cat.codes
test_cal_features = cal.values.T




train_cal_features = test_cal_features[:,:-single_prediction_length]


test_cal_features_list = [test_cal_features] * len(sales_train_evaluation)
train_cal_features_list = [train_cal_features] * len(sales_train_evaluation)

We then go on to build static features (features which are constant and series-specific). Here, we make use of all categorical features that are provided to us as part of the M5 data.

In [5]:
state_ids = sales_train_validation["state_id"].astype('category').cat.codes.values
state_ids_un , state_ids_counts = np.unique(state_ids, return_counts=True)

store_ids = sales_train_validation["store_id"].astype('category').cat.codes.values
store_ids_un , store_ids_counts = np.unique(store_ids, return_counts=True)

cat_ids = sales_train_validation["cat_id"].astype('category').cat.codes.values
cat_ids_un , cat_ids_counts = np.unique(cat_ids, return_counts=True)

dept_ids = sales_train_validation["dept_id"].astype('category').cat.codes.values
dept_ids_un , dept_ids_counts = np.unique(dept_ids, return_counts=True)

item_ids = sales_train_validation["item_id"].astype('category').cat.codes.values
item_ids_un , item_ids_counts = np.unique(item_ids, return_counts=True)

stat_cat_list = [item_ids, dept_ids, cat_ids, store_ids, state_ids]

stat_cat = np.concatenate(stat_cat_list)
stat_cat = stat_cat.reshape(len(stat_cat_list), len(item_ids)).T

stat_cat_cardinalities = [len(item_ids_un), len(dept_ids_un), len(cat_ids_un), len(store_ids_un), len(state_ids_un)]
embeddings=list(np.array(stat_cat_cardinalities)+2)

Finally, we can build both the training and the testing set from target values and both static and dynamic features.

In [6]:
from gluonts.dataset.common import load_datasets, ListDataset
from gluonts.dataset.field_names import FieldName
train_df = sales_train_evaluation.drop(["id","item_id","dept_id","cat_id","store_id","state_id"], axis=1)
train_target_values = train_df.values

train_val_df = sales_train_validation.drop(["id","item_id","dept_id","cat_id","store_id","state_id"], axis=1)
test_target_values = [np.append(ts, np.ones(submission_prediction_length) * np.nan) for ts in train_val_df.values]

m5_dates = [pd.Timestamp("2011-01-29", freq='1D') for _ in range(len(sales_train_validation))]

train_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start,
        FieldName.FEAT_DYNAMIC_REAL: fdr,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, start, fdr, fsc) in zip(train_target_values,
                                         m5_dates,
                                         train_cal_features_list,
                                         stat_cat)
], freq="D")

test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start,
        FieldName.FEAT_DYNAMIC_REAL: fdr,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, start, fdr, fsc) in zip(test_target_values,
                                         m5_dates,
                                         test_cal_features_list,
                                         stat_cat)
], freq="D")

Just to be sure, we quickly verify that dataset format is correct and that our dataset does indeed contain the correct target values as well as dynamic and static features.

In [7]:
next(iter(train_ds))

{'target': array([0., 0., 0., ..., 3., 0., 1.], dtype=float32),
 'start': Timestamp('2011-01-29 00:00:00', freq='D'),
 'feat_dynamic_real': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'feat_static_cat': array([1437,    3,    1,    0,    0], dtype=int32),
 'source': SourceContext(source='list_data', row=1)}

#### Define the estimator

Having obtained our training and testing data, we can now create a GluonTS estimator. We will use the `DeepAREstimator`, an autoregressive RNN which was developed primarily for the purpose of time series forecasting. Also, since GluonTS is mainly target at probabilistic time series forecasting, lots of different output distributions can be specified. In the M5 case, we think that the `NegativeBinomialOutput` distribution best describes the output, because it models count data quite well.

For a full list of available estimators and possible initialization arguments see https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.html.

For a full list of available output distributions and possible initialization arguments see https://gluon-ts.mxnet.io/api/gluonts/gluonts.distribution.html.

Due to the stochastic nature of the network, we train the network 5 times and average the result. Even so, the results are not fully repreducable.

##### Network design
- DeepAR network
- `NegativeBinomialOutput` distribution
- 3 layers with 30 cells each
- LSTM cells
- drop out rate 0.1
- learning rate 1e-3
- epochs  100
- num_batches_per_epoch 1000,
- batch_size=64,
- weight_decay=1e-8

In [8]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from pathlib import Path

model_name="m_3_40_d01_final"

predictions=[]

for i in range(5):
    p="./models/"+model_name+"/t"+str(i)
    Path(p).mkdir(parents=True, exist_ok=True)
    estimator = DeepAREstimator(
        prediction_length=prediction_length,
        freq="D",
        distr_output = NegativeBinomialOutput(),
        use_feat_dynamic_real=True,
        use_feat_static_cat=True,
        cardinality=stat_cat_cardinalities,
        embedding_dimension=embeddings,
        num_layers=3,
        num_cells=40,
        cell_type='lstm',
        dropout_rate=0.1,
        num_parallel_samples=200,
       # lags_seq=[7,28],
        trainer=Trainer(
            learning_rate=1e-3,
            epochs=100,
            num_batches_per_epoch=1000,
            batch_size=64,
           # patience=6,
            weight_decay=1e-8
        ),

    ) 
    #train
    predictor = estimator.train(train_ds)
    
    predictor.serialize(Path(p))
    
    #predict
    

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,
        predictor=predictor,
        num_samples=400
    )

    print("Obtaining time series conditioning values ...")
    tss = list(tqdm(ts_it, total=len(test_ds)))
    print("Obtaining time series predictions ...")
    forecasts = list(tqdm(forecast_it, total=len(test_ds)))
    
    forecasts_acc = np.zeros((len(forecasts), prediction_length))
    for i in range(len(forecasts)):
        forecasts_acc[i] = np.mean(forecasts[i].samples, axis=0)
        
    # reshape
    forecasts_acc_sub = np.zeros((len(forecasts)*2, single_prediction_length))
    forecasts_acc_sub[:len(forecasts)] = forecasts_acc[:,:single_prediction_length]
    forecasts_acc_sub[len(forecasts):] = forecasts_acc[:,single_prediction_length:]
    
    predictions.append(forecasts_acc_sub)

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/1000 [00:00<?, ?it/s]INFO:numexpr.utils:Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:root:Number of parameters in DeepARTrainingNetwork: 9835434
100%|██████████| 1000/1000 [01:59<00:00,  8.38it/s, avg_epoch_loss=0.894]
INFO:root:Epoch[0] Elapsed time 119.279 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=0.893813
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 1000/1000 [01:58<00:00,  8.41it/s, avg_epoch_loss=0.868]
INFO:root:Epoch[1] Elapsed time 118.915 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.868275
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 1000/1000 [01:58<00:00,  8.40it/s, avg_epoch_loss=0.867]
INFO:root:Epoch[2] Elapsed time 118.999 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_lo

INFO:root:Epoch[35] Learning rate is 0.001
100%|██████████| 1000/1000 [01:59<00:00,  8.36it/s, avg_epoch_loss=0.844]
INFO:root:Epoch[35] Elapsed time 119.605 seconds
INFO:root:Epoch[35] Evaluation metric 'epoch_loss'=0.844110
INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 1000/1000 [01:58<00:00,  8.41it/s, avg_epoch_loss=0.837]
INFO:root:Epoch[36] Elapsed time 118.959 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=0.836515
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 1000/1000 [01:59<00:00,  8.39it/s, avg_epoch_loss=0.836]
INFO:root:Epoch[37] Elapsed time 119.128 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=0.835780
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 1000/1000 [02:00<00:00,  8.33it/s, avg_epoch_loss=0.841]
INFO:root:Epoch[38] Elapsed time 120.100 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=0.840666
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 1000/1000 [01:59<00:00,  8.40it/s, a

INFO:root:Epoch[70] Evaluation metric 'epoch_loss'=0.838356
INFO:root:Epoch[71] Learning rate is 0.00025
100%|██████████| 1000/1000 [01:57<00:00,  8.48it/s, avg_epoch_loss=0.843]
INFO:root:Epoch[71] Elapsed time 117.965 seconds
INFO:root:Epoch[71] Evaluation metric 'epoch_loss'=0.843335
INFO:root:Epoch[72] Learning rate is 0.00025
100%|██████████| 1000/1000 [01:59<00:00,  8.36it/s, avg_epoch_loss=0.837]
INFO:root:Epoch[72] Elapsed time 119.619 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=0.837102
INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 1000/1000 [01:59<00:00,  8.34it/s, avg_epoch_loss=0.837]
INFO:root:Epoch[73] Elapsed time 119.957 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=0.836668
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 1000/1000 [01:59<00:00,  8.38it/s, avg_epoch_loss=0.835]
INFO:root:Epoch[74] Elapsed time 119.273 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=0.834883
INFO:root:Loading parameters

Obtaining time series conditioning values ...



Obtaining time series predictions ...


...

INFO:root:Using GPU
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 1000/1000 [02:04<00:00,  8.01it/s, avg_epoch_loss=0.887]
INFO:root:Epoch[0] Elapsed time 124.821 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=0.887169
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 1000/1000 [02:04<00:00,  8.01it/s, avg_epoch_loss=0.868]
INFO:root:Epoch[1] Elapsed time 124.871 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.867777
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 1000/1000 [02:03<00:00,  8.09it/s, avg_epoch_loss=0.868]
INFO:root:Epoch[2] Elapsed time 123.559 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=0.868094
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 1000/1000 [02:04<00:00,  8.01it/s, avg_epoch_loss=0.857]
INFO:root:Epoch[3] Elapsed time 124.778 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=0.857201
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████

100%|██████████| 1000/1000 [02:04<00:00,  8.04it/s, avg_epoch_loss=0.84]
INFO:root:Epoch[35] Elapsed time 124.353 seconds
INFO:root:Epoch[35] Evaluation metric 'epoch_loss'=0.840457
INFO:root:Epoch[36] Learning rate is 0.0005
100%|██████████| 1000/1000 [02:04<00:00,  8.04it/s, avg_epoch_loss=0.839]
INFO:root:Epoch[36] Elapsed time 124.430 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=0.839176
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 1000/1000 [02:04<00:00,  8.01it/s, avg_epoch_loss=0.84]
INFO:root:Epoch[37] Elapsed time 124.877 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=0.839725
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 1000/1000 [02:03<00:00,  8.07it/s, avg_epoch_loss=0.836]
INFO:root:Epoch[38] Elapsed time 123.958 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=0.836280
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 1000/1000 [02:05<00:00,  8.00it/s, avg_epoch_loss=0.836]
INFO:root:Epoch[39] 

100%|██████████| 1000/1000 [02:04<00:00,  8.06it/s, avg_epoch_loss=0.843]
INFO:root:Epoch[70] Elapsed time 124.142 seconds
INFO:root:Epoch[70] Evaluation metric 'epoch_loss'=0.843220
INFO:root:Epoch[71] Learning rate is 6.25e-05
100%|██████████| 1000/1000 [02:04<00:00,  8.05it/s, avg_epoch_loss=0.837]
INFO:root:Epoch[71] Elapsed time 124.230 seconds
INFO:root:Epoch[71] Evaluation metric 'epoch_loss'=0.837048
INFO:root:Epoch[72] Learning rate is 6.25e-05
100%|██████████| 1000/1000 [02:04<00:00,  8.02it/s, avg_epoch_loss=0.836]
INFO:root:Epoch[72] Elapsed time 124.732 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=0.835673
INFO:root:Epoch[73] Learning rate is 6.25e-05
100%|██████████| 1000/1000 [02:04<00:00,  8.04it/s, avg_epoch_loss=0.845]
INFO:root:Epoch[73] Elapsed time 124.316 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=0.844760
INFO:root:Epoch[74] Learning rate is 6.25e-05
100%|██████████| 1000/1000 [02:05<00:00,  7.99it/s, avg_epoch_loss=0.837]
INFO:root:

Obtaining time series conditioning values ...



Obtaining time series predictions ...


In [9]:
predictions[0].shape

(60980, 28)

In [10]:
forecasts_acc_sub=np.mean(np.array(predictions),axis=0)

forecasts_acc_sub.shape

(60980, 28)

In [12]:

import time

sample_submission = pd.read_csv(f'{m5_input_path}/sample_submission.csv')
sample_submission.iloc[:,1:] = forecasts_acc_sub

submission_id = 'submission_{}.csv'.format(int(time.time()))
submission_id  ="submission_gluonts_m40_final.csv"
sample_submission.to_csv(submission_id, index=False)